In [1]:
from glob import glob
import json

In [2]:
files = glob('texts_0.jsonl*.splitted.requested')
files

['texts_0.jsonl08.splitted.requested',
 'texts_0.jsonl02.splitted.requested',
 'texts_0.jsonl09.splitted.requested',
 'texts_0.jsonl10.splitted.requested',
 'texts_0.jsonl03.splitted.requested',
 'texts_0.jsonl07.splitted.requested',
 'texts_0.jsonl05.splitted.requested',
 'texts_0.jsonl14.splitted.requested',
 'texts_0.jsonl12.splitted.requested',
 'texts_0.jsonl13.splitted.requested',
 'texts_0.jsonl06.splitted.requested',
 'texts_0.jsonl01.splitted.requested',
 'texts_0.jsonl04.splitted.requested',
 'texts_0.jsonl00.splitted.requested',
 'texts_0.jsonl11.splitted.requested']

In [3]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue
            mapping[l['src']] = l['r']['result']
            
len(mapping)

1498726

In [4]:
all_data = []
with open('train_0.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data = []
        for d_ in l['data']:
            data.append({
                'value': d_,
                'value_ms': mapping.get(d_)
            })
        all_data.append(data)

In [5]:
all_data[0]

[{'value': 'How can cross training benefit groups like runners, swimmers, or weightlifters?',
  'value_ms': 'Bagaimanakah latihan silang boleh memberi manfaat kepada kumpulan seperti pelari, perenang atau atlet angkat berat?'},
 {'value': 'Cross training can benefit groups like runners, swimmers, or weightlifters in the following ways:\n\n1. Reduces the risk of injury: Cross training involves different types of exercises that work different muscle groups. This reduces the risk of overuse injuries that may result from repetitive use of the same muscles.\n\n2. Improves overall fitness: Cross training helps improve overall fitness levels by maintaining a balance of strength, endurance, flexibility, and cardiovascular fitness.\n\n3. Breaks monotony: Cross training adds variety to your fitness routine by introducing new exercises, which can help you stay motivated and avoid boredom that often comes with doing the same exercises repeatedly.\n\n4. Increases strength: Cross training helps in b

In [6]:
inside = []
with open('train_0.jsonl.translated') as fopen:
    for l in fopen:
        if 'model bahasa' in l or 'openai' in l:
            inside.append(l)
            if len(inside) >= 10:
                break

In [7]:
break_at = [
    'help.openai.com',
    'openai'
]

In [8]:
from tqdm import tqdm
import random

all_texts = []
for i in tqdm(range(len(all_data))):
    texts = []
    for no, d in enumerate(all_data[i]):
        
        if no % 2 == 0:
            t = '<manusia>: '
            if random.random() > 0.5:
                c = 'value'
            else:
                c = 'value_ms'
        else:
            t = '<bot>: '
                
        if d[c] is None:
            break
        
        if any([b in d[c].lower() for b in break_at]):
            print(c[c], '\n-----\n')
            break
        
        n = d[c]
        n = n.strip()
        if len(n) < 3:
            break
        t = t + n
        texts.append(t)

    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    while len(texts) and texts[-1].startswith('<manusia>'):
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|████████████████████████████████| 150000/150000 [00:03<00:00, 45961.72it/s]


In [9]:
len(all_texts)

149994

In [10]:
all_texts[0]

"<manusia>: How can cross training benefit groups like runners, swimmers, or weightlifters?\n<bot>: Cross training can benefit groups like runners, swimmers, or weightlifters in the following ways:\n\n1. Reduces the risk of injury: Cross training involves different types of exercises that work different muscle groups. This reduces the risk of overuse injuries that may result from repetitive use of the same muscles.\n\n2. Improves overall fitness: Cross training helps improve overall fitness levels by maintaining a balance of strength, endurance, flexibility, and cardiovascular fitness.\n\n3. Breaks monotony: Cross training adds variety to your fitness routine by introducing new exercises, which can help you stay motivated and avoid boredom that often comes with doing the same exercises repeatedly.\n\n4. Increases strength: Cross training helps in building strength by incorporating exercises that target different muscle groups. This helps you build strength in areas that may be underdev

In [11]:
for t in all_texts:
    splitted = t.split('<manusia>:')
    if '<bot>:' not in splitted[-1]:
        print(t)
        break

In [12]:
with open('part0-alternate_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')